In [36]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset="train",shuffle = True)

In [28]:
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [29]:
print("\n".join(twenty_train.data[0].split("\n")))


From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







### Extracting features from text files
- Use Bag of Words model in this example
- In BoW, each unique word in the dictionary will correspond to a feature.

#### Convert words to vectors

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 130107)

#### Build the trainning model with TF and TF-IDF

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
x_train_tfidf.shape

(11314, 130107)

### Running ML algorithms
- Build a 'Naive Bayes' classifier (algorithm used for text classification)

In [32]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train_tfidf, twenty_train.target)

#### Building a pipeline: We can write less code and do all of the above, by building a pipeline as follows:

In [33]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect',CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf',MultinomialNB())])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [34]:
import numpy as np
twenty_test = fetch_20newsgroups(subset = "test", shuffle =True)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.7738980350504514

In [38]:
from sklearn.linear_model import SGDClassifier

text_clf_svm = Pipeline([('vect',CountVectorizer()),
                        ('tfidf',TfidfTransformer()),
                        ('clf-svm',SGDClassifier(loss = 'hinge',penalty='l2',alpha=1e-3,
                                                 n_iter_no_change =5,random_state =42)),
                         ])

_ = text_clf_svm.fit(twenty_train.data,twenty_train.target)

predicted_svm = text_clf_svm.predict(twenty_test.data)
np.mean(predicted_svm == twenty_test.target)

0.8240839086563994

### Grid Search

In [53]:
from sklearn.model_selection import GridSearchCV
parameters ={
    'vect__ngram_range':[(1,1),(1,2)],
    'tfidf__use_idf':(True,False),
    'clf__alpha':(1e-2,1e-3)
}

In [55]:
gs_clf = GridSearchCV(text_clf,parameters,n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

c:\users\wang_cheng\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


MemoryError: 

In [ ]:
print(gs_clf.best_score_)
print(gs_clf.best_params_)